In [ ]:
# Used for selecting the 6 best features per cluster. 
# We're using mean squared error of each variable vs. the ALSFRS_score, and take the best 6. 

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import vectorizer_beta
from sklearn import linear_model

In [3]:
proact_data = pd.read_csv('../train_data.csv', sep = '|', index_col=False)
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col=False)
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col=False)
X = pd.merge(clusters, proact_data, on = "SubjectID")
Y = pd.merge(clusters, slope, on = "SubjectID")
print Y.shape, X.shape, clusters.shape
X.head()

(1777, 3) (1138647, 7) (1777, 2)


,SubjectID,cluster,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,0,Demographic,Gender,F,NaN,0.0
1,533,0,Demographic,Age,65,NaN,0.0
2,533,0,Demographic,Race,White,NaN,0.0
3,533,0,ALSHX,onset_delta,-1023,NaN,0.0
4,533,0,ALSHX,diag_delta,-44,NaN,0.0


In [9]:
from vectorizer_beta import * 
best_features_per_cluster = {}

for c in clusters['cluster'].unique():
    seg_X, seg_Y = X[X['cluster'] == c], Y[Y['cluster'] == c]
    score_per_feature = {}
    for feature_name, func in func_per_feature.iteritems():
        seg_X_fam = list(func_per_feature[feature_name])[0](seg_X, feature_name)
        seg_Y_fam = pd.merge(seg_Y, seg_X_fam, left_on = 'SubjectID', right_index = True, how='left')
        seg_Y_fam = seg_Y_fam.fillna(seg_Y_fam.mean())
        regr = linear_model.LinearRegression()
        seg_X_fam = seg_Y_fam.drop('ALSFRS_slope', 1)
        regr.fit(seg_X_fam, seg_Y_fam['ALSFRS_slope'])
        score_per_feature[feature_name] = np.mean((regr.predict(seg_X_fam) - seg_Y_fam['ALSFRS_slope']) ** 2)
    print c, score_per_feature
    best_features_per_cluster[c] = sorted(score_per_feature, key=score_per_feature.get)[:6]

    ### DUMMY!!!
    best_features_per_cluster[c] = [
        'ALSFRS_Total_last_zscore',
        'ALSFRS_Total_slope_mean_zscore',
        'weight_mean_zscore', 
        'weight_pct_diff_zscore',
        'Age_last_zscore'
    ]


    
best_features_per_cluster

0 {'weight': 0.36580549782154553, 'bp_systolic': 0.3872706334991971, 'Gender': 0.38843806436890316, 'Age': 0.38828996983124436, 'BMI': 0.38829637391520883, 'bp_diastolic': 0.38513664036749956, 'pulse': 0.37281137100278622, 'respiratory_rate': 0.38069215217549818, 'Race': 0.38745996187703152, 'height': 0.38795863808372805, 'ALSFRS_Total': 0.15575957187074072, 'temperature': 0.38655944171153278}
1 {'weight': 0.33779330206477698, 'bp_systolic': 0.39043121666603303, 'Gender': 0.39406157038531719, 'Age': 0.39406395532147043, 'BMI': 0.39387843333155559, 'bp_diastolic': 0.38787680706659805, 'pulse': 0.38215353289649079, 'respiratory_rate': 0.38748792268630783, 'Race': 0.39127186894437682, 'height': 0.39269322969645842, 'ALSFRS_Total': 0.11921953292698396, 'temperature': 0.39237852618255414}


{0: ['ALSFRS_Total_last_zscore',
  'ALSFRS_Total_slope_mean_zscore',
  'weight_mean_zscore',
  'weight_pct_diff_zscore',
  'Age_last_zscore'],
 1: ['ALSFRS_Total_last_zscore',
  'ALSFRS_Total_slope_mean_zscore',
  'weight_mean_zscore',
  'weight_pct_diff_zscore',
  'Age_last_zscore']}

In [10]:
import pickle 
with open("best_features_per_cluster.pickle", "wb") as output_file:
    pickle.dump(best_features_per_cluster, output_file)
